# 文書分類ハンズオン

## 機械学習モデル開発のワークフローと本章で扱う内容

なお、動作確認は以下の環境で行いました。

- Machine (AWS EC2 p2.xlargeインスタンス)
    - OS: Ubuntu 16.04
    - CPU: Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
    - RAM: 64GB 
    - GPU: NVIDIA Tesla K80
- Python
    - Python 3.7
    - PyTorch 1.3

## 文書分類ハンズオン

### 本章で扱う自然言語処理ツールの解説

#### Transformers

Transformersで用いることのできるモデルのリストはHugging Faceのホームページ (https://huggingface.co/models) にて公開されています。

-  `bert-base-japanese`:
-  `bert-base-japanese-whole-word-masking`
-  `bert-base-japanese-char`
-  `bert-base-japanese-char-whole-word-masking`

Transformersは、FaceBookが開発しているPyTorchおよび、GoogleのTensorFlowのバージョン2.0以降に対応しています。ここではPyTorchを用いることにします。

In [ ]:
# 環境によってインストールコマンドが異なります。https://pytorch.org/get-started/を参照してください。
# 2019年12月現在、NVIDIAのGPUを搭載したLinuxマシンにAnacondaでPyTorchをインストールするコマンドは以下の通りです。
!conda install pytorch torchvision cudatoolkit=10.1 -c pytorch
# LinuxでかつGPUがない場合は !conda install pytorch torchvision cpuonly -c pytorch

In [15]:
!pip install transformers

`bert-base-japanese*` は形態素解析のライブラリとして内部でMeCabを用いているので、`mecab-python3` もインストールしておきましょう。

In [ ]:
!pip install mecab-python3

また、以降で補助的に利用するライブラリもインストールしてください。

In [18]:
!pip install pandas scikit-learn seaborn

     |████████████████████████████████| 215kB 3.1MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/61/42/3e92d7aa64295483fbca20a86c89b34d0cb43cffaadaffe028793902d790/matplotlib-3.1.2-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/c0/0c/fc2e007d9a992d997f04a80125b0f183da7fb554f1de701bbb70a8e7d479/pyparsing-2.4.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/93/f8/518fb0bb89860eea6ff1b96483fbd9236d5ee991485d0f3eceff1770f654/kiwisolver-1.1.0-cp37-cp37m-manylinux1_x86_64.whl


- pandas
- scikit-learn
- seaborn

BERTを用いて日本語の文章を分類する手順はおおまかに書くと以下のようになります。

1. aaa
2. aaa



In [2]:
from transformers import BertJapaneseTokenizer

In [8]:
tokenizer = BertJapaneseTokenizer.from_pretrained('bert-base-japanese-whole-word-masking')
tokenizer.tokenize('いつもプレゼンテーションの撮影に無音カメラアプリをご利用いただきありがとうございます。')

['いつも',
 'プレゼンテーション',
 'の',
 '撮影',
 'に',
 '無',
 '##音',
 'カメラ',
 '##アプリ',
 'を',
 'ご',
 '利用',
 'いただ',
 '##き',
 'ありがとう',
 'ござい',
 'ます',
 '。']

### データセットの準備
#### livedoor ニュースコーパス

今回は日本語における自然言語処理の試験用データセットとしてしばしば用いられる「livedoor ニュースコーパス」を用います。

livedoorニュースはもともと株式会社ライブドアが運営するニュースサイトでしたが、株式会社ライブドアが旧ハンゲームジャパン株式会社であるNHN Japan株式会社に買収され、現在はNHN Japanが社名変更したLINE株式会社により運営されています。livedoorニュースの記事の一部には「クリエイティブ・コモンズライセンス『表示 – 改変禁止』」が適用されており、営利目的を含めて再配布可能となっています。該当するニュース記事を2012年9月上旬に株式会社ロンウイットが収集し、HTMLタグの除去などクリーニングを施した状態で公開しているのが「livedoor ニュースコーパス」です。

livedoor ニュースコーパスは以下のリンクよりダウンロード可能です。

https://www.rondhuit.com/download.html#ldcc

オープンソースの全文検索システムApache Solrで扱いやすいようXML形式でニュースが格納されている `livedoor-news-data.tar.gz` と、シンプルに各々のニュースをテキストファイルとして扱っている `ldcc-20140209.tar.gz` が公開されています。

今回は後者の `ldcc-20140209.tar.gz` をダウンロードしてください。`tar xzvf ldcc-20140209.tar.gz` などにより解凍すると `text` という名前のディレクトリが出てきます。以下のPythonスクリプトを実行するとコーパスのダウンロードと圧縮ファイルの解凍が行われ、カレントディレクトリに `text` ディレクトリが作成されます。

In [4]:
import os
import urllib.request
import tarfile

# dataディレクトリの作成
#os.makedirs('data', exist_ok=True)

url = 'https://www.rondhuit.com/download/ldcc-20140209.tar.gz'
file_name = 'ldcc-20140209.tar.gz'

# dataディレクトリへのlivedoor ニュースコーパスのダウンロードと解凍
if not os.path.exists(file_name):
    urllib.request.urlretrieve(url, file_name)
    # tar.gzファイルを読み込み
    with tarfile.open(file_name) as tar:
        tar.extractall()
    # tar.gzファイルを消去
    os.remove(file_name)

`text` ディレクトリの中身の構造は以下の通りです。

```
text
├── CHANGES.txt
├── README.txt
├── dokujo-tsushin
├── it-life-hack
├── kaden-channel
├── livedoor-homme
├── movie-enter
├── peachy
├── smax
├── sports-watch
└── topic-news
```

`dokujo-tsushin` から `topic-news` はディレクトリであり、それぞれにニュース記事のテキストが格納されています。

```
text
├── CHANGES.txt
├── README.txt
├── dokujo-tsushin
│   ├── LICENSE.txt
│   ├── dokujo-tsushin-4778030.txt
│   ├── dokujo-tsushin-4778031.txt
│   ├── dokujo-tsushin-4782522.txt
...（以下略）
```

ニュース提供元は以下の9つです。記事の本文だけを見て、その記事がどのカテゴリに属しているのか（独女通信のニュースなのか、ITライフハックのニュースなのか、など）を判別する文書分類モデルを作成するのが本章の目的です。

- 独女通信 (http://news.livedoor.com/category/vender/90/)
- ITライフハック (http://news.livedoor.com/category/vender/223/)
- 家電チャンネル (http://news.livedoor.com/category/vender/kadench/)
- livedoor HOMME (http://news.livedoor.com/category/vender/homme/)
- MOVIE ENTER (http://news.livedoor.com/category/vender/movie_enter/)
- Peachy (http://news.livedoor.com/category/vender/ldgirls/)
- エスマックス (http://news.livedoor.com/category/vender/smax/)
- Sports Watch (http://news.livedoor.com/category/vender/208/)
- トピックニュース (http://news.livedoor.com/category/vender/news/)

ちなみに、上記サービスのうちいくつかはドメインが変わっていたり終了しているので一部リンクが切れています。それぞれの記事ファイル（dokujo-tsushin-4778030.txtなど）は以下のフォーマットで構成されています。

- １行目: 記事のURL
- ２行目: 記事の日付
- ３行目: 記事のタイトル
- ４行目以降： 記事の本文

このままでは少し扱いづらいのでひとつのtsv (tab-separated values) にまとめます。

In [26]:
services = [
    'dokujo-tsushin',
    'it-life-hack',
    'kaden-channel',
    'livedoor-homme',
    'movie-enter',
    'peachy',
    'smax',
    'sports-watch',
    'topic-news'
]
index = ['url', 'datetime', 'title', 'body']

In [41]:
import os
import glob

import pandas as pd

df = pd.DataFrame()

for service in services:
    print('processing ' + service)
    wild_card = os.path.join('text', service, '*.txt')
    file_paths = glob.glob(wild_card)
    for file_path in file_paths:
        with open(file_path, 'r') as f:
            lines = f.readlines()
            series_dict = {'service': service}
            for num, line in enumerate(lines):
                # 改行を削除
                line = line.replace('\n', '')
                if num < len(index):
                    series_dict[index[num]] = line
                else:
                    series_dict['body'] += line
            
            s = pd.Series(series_dict)
            df = df.append(s, ignore_index=True)
print('done')         

processing dokujo-tsushin
processing it-life-hack
processing kaden-channel
processing livedoor-homme
processing movie-enter
processing peachy
processing smax
processing sports-watch
processing topic-news
done


In [42]:
df.head()

,body,datetime,service,title,url
0,「しっかりメイクをしている顔よりも、スッピンのほうがいい」と言う男性は少なくない。しかしメイ...,2011-10-13T15:28:00+0900,dokujo-tsushin,「スッピンのほうがいい」って本当？——言葉の裏に隠された男の本音,http://news.livedoor.com/article/detail/5934284/
1,多くの女性が悩む「お腹の張り」。ファッション誌をめくる度に「モデルさんは細くていいなあ…」...,2010-06-25T14:00:00+0900,dokujo-tsushin,まるで妊婦！？ モデルの7割が悩む「ポッコリお腹」,http://news.livedoor.com/article/detail/4848785/
2,実在した武士の家計簿を元に制作された映画「武士の家計簿」。家計簿をつけ収入収支を管理し、誠実...,2011-01-04T14:00:00+0900,dokujo-tsushin,つける？ つけない？ 独女の家計簿,http://news.livedoor.com/article/detail/5247353/
3,早いもので、もうすぐ季節は夏。今年は東日本を中心に、ビジネスシーンでのクールビズを例年以上に...,2011-06-22T12:30:00+0900,dokujo-tsushin,夏に多発！下着チラみせ独女はアリかナシか？,http://news.livedoor.com/article/detail/5653189/
4,女に必要なものは何か。一寸先は真っ暗闇の、生きるというより、生き抜かなくてはいけないこのご時...,2012-01-24T16:09:00+0900,dokujo-tsushin,東京23区女ひとり風呂 vol.04「文京区・銭湯後はこの一杯！」Presented by ...,http://news.livedoor.com/article/detail/6218046/


In [43]:
df.to_csv('news.tsv', sep='\t')

#### ラベリング

### 前処理

#### 形態素解析
#### ストップワード除去

### 文書分類モデル





In [10]:
from transformers import BertForSequenceClassification
net = BertForSequenceClassification.from_pretrained('bert-base-japanese-whole-word-masking', num_labels=9)

このニューラルネットワークの構造をnetron (https://github.com/lutzroeder/netron) というツールを用いて可視化すると次のようになります。
<img src="../figures/bert_classifier_netron.png" alt="bert_classifier_netron" width="150">

### 評価と比較

### モデルのデプロイ

## まとめ